In [ ]:
import io
import os
import pandas as pd
import re
#open vcf as dataframe
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'chrom' })

#read clinvar for GrCH38 build
clin = read_vcf('/project/InternalMedicine/Kong_lab/shared/workspace/CSV/annotation/clinvar_hg38.vcf')

In [ ]:
#split by ;, have to look for entry fields by regex
with open('/project/InternalMedicine/Kong_lab/shared/workspace/CSV/annotation/clinvar_hg38.vcf', 'r') as f:
    head1 = []
    for line in f:
        if line.startswith('##INFO=<ID='):
            j = re.findall('##INFO=<ID=(.*),Num', line)
            head1 +=j

In [ ]:
#now look for values
for j in head1:
    clin[j] = ''
    for i in range(len(clin)):
        var = '{}=([A-Za-z0-9._-|:>,]+)'
        d = re.search(var.format(j),clin.INFO[i])
        if d is not None :
            clin.loc[i, j] = d.group(1)

In [ ]:
#merge info and original df, drop original clumped info columns
clinvar = clin.drop(['INFO'], axis=1)
#write to csv
clinvar.to_csv('/project/InternalMedicine/Kong_lab/shared/workspace/CSV/annotation/clinvar_hg38.csv', index=False)

In [ ]:
#LS gene variants
LS=clinvar[~pd.isna(clinvar['GENEINFO'])&clinvar['GENEINFO'].str.contains('PMS2|MSH2|MSH6|MLH1')&clinvar['CLNSIG'].isin(['Pathogenic','Likely_pathogenic'])]


In [ ]:
#more confident filtering
mult_review= LS[LS['CLNREVSTAT'].isin(['reviewed_by_expert_panel','criteria_provided,_multiple_submitters,_no_conflicts'])]
mult_review.to_csv('jiheum_var_2.csv',index=False)


In [ ]:
#filter for potential pms2cl variants in exons 12-15
pms2 = mult_review[mult_review['GENEINFO'].str.contains('PMS2')]

pms2cl = pms2[pms2['POS']<5983000]

pms2cl[~pms2cl['MC'].str.contains('acceptor')].to_csv('pms2_pseudo.csv',index=False)